#### Hard Attention

El mecanismo de Hard Attention a diferencia de su contraparte "Soft Attention" que permite que las gradientes sean propagadas hacia atrás de manera suave al asignar pesos a todas las partes de la entrada. Hard Attention se enfoca selectivamente en partes particulares de los datos de entrada, ignorando el resto. Este enfoque selectivo es parecido a la manera en que los humanos prestán atención a aspectos particulares de una escena visual o pieza de texto mientras ignoran otros.

### Características clave  
- **Selectividad**: El mecanismo de Hard Attention es selectivo, enfocándose en partes particulares de los datos de entrada. Por ejemplo, en etiquetado de imágenes, el modelo podría enfocarse en objetos en especifico en una imagen en cada paso de la generación de la etiqueta.  
  
- **No diferenciabilidad**: Hard Attention involucra hacer decisiones discretas, es inherentemente no diferenciable. Esta característica plantea desafios en el entrenamiento usando tecnicas de propagación hacia atrás, que depende de optimización basada en gradientes.  
  
- **Estocasticidad**: El proceso de selección de Hard Attention es seguido estocastico, significando que involucra cierto nivel de aleatoriedad. Los modelos podrían usar técnicas como muestreo o aprendizaje reforzado para decidir que partes de la entrada enfocarse.  
  
- **Eficiencia**: Al enfocarse solo en las partes relevantes de la entrada, los mecanismos de Hard Attention pueden ser más eficientes, especialmente con entradas grandes. Ayudan con la carga computacional de procesar y asignar pesos a todas las partes de la entrada en cada paso como lo hace Soft Attention.

#### Retos de entrenamiento

Dada la naturaleza no diferenciable de Hard Attention, entrenar modelos que usan este mecanismo puede ser desafiante. Algunas técnicas comunes para abordar este desafío incluyen:

- **Muestreo**: En lugar de hacer una selección dura, los modelos pueden muestrear de una distribución de probabilidad para seleccionar partes de la entrada. Esto introduce estocasticidad en el proceso de selección y permite que las gradientes sean propagadas hacia atrás. 

- **Aprendizaje reforzado**: Los modelos pueden ser entrenados usando técnicas de aprendizaje reforzado, donde se recompensa o penaliza la selección de partes de la entrada basado en el desempeño del modelo. Esto puede ser útil cuando se necesita aprender una política de selección óptima.

- **Gumbel-Softmax Trick**: Esta técnica involucra muestrear de una distribución de Gumbel-Softmax, que es una aproximación suave de la distribución de Gumbel. Esto permite que las gradientes sean propagadas hacia atrás y es útil para entrenar modelos con selección dura. 

- **Monte Carlo Methods**: En lugar de hacer una selección dura, los modelos pueden usar métodos de Monte Carlo para estimar la selección óptima. Esto puede ser útil cuando se necesita una aproximación más suave a la selección dura.

### Formulas y Algoritmos

Hard Attention no sigue una fórmula o algoritmo específico, ya que la selección de partes de la entrada es determinada por el modelo en cada paso. Sin embargo, algunos enfoques comunes para implementar Hard Attention incluyen: 

- **Selección Dura**: En este enfoque, el modelo selecciona una parte particular de la entrada en cada paso. Esto puede ser implementado usando técnicas como muestreo o aprendizaje reforzado. Como por ejemplo en el caso de etiquetado de imágenes, el modelo podría enfocarse en diferentes regiones de la imagen en cada paso de la generación de la etiqueta. 

- **Muestreo Estocástico**: En lugar de hacer una selección dura, el modelo puede muestrear de una distribución de probabilidad para seleccionar partes de la entrada. Esto introduce estocasticidad en el proceso de selección y permite que las gradientes sean propagadas hacia atrás.

- **Función de decisión**: Los modelos que usan Hard Attention necesitan una función de decisión para determinar que partes de la entrada enfocarse en cada paso. Esta función puede ser aprendida durante el entrenamiento usando técnicas como aprendizaje reforzado o muestreo. 

- **Salida del modelo**: La salida del modelo en cada paso depende de las partes de la entrada que son seleccionadas por el mecanismo de Hard Attention. Por ejemplo, en el caso de generación de texto, la salida del modelo en cada paso podría depender de las palabras seleccionadas de la entrada.

### Ejemplo de aplicación
#### Tarea de procesamiento de secuencia con Hard Attention

In [21]:
import torch
import torch.nn as nn

class HardAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(HardAttention, self).__init__()
        # Transforma la dimensión de entrada a la dimensión oculta
        self.hidden_layer = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.Tanh()
        # Califica cada vector de dimensión oculta (de hidden_dim a 1)
        self.scoring_function = nn.Linear(hidden_dim, 1)

    def forward(self, sequence):
        """
        sequence: Un lote de secuencias, forma (tamaño_lote, longitud_secuencia, dim_entrada)
        """
        # Pasa la entrada a través de una capa oculta y una función de activación
        hidden_repr = self.activation(self.hidden_layer(sequence))
        # Calcula las puntuaciones para cada vector en la secuencia (la forma cambia a (tamaño_lote, longitud_secuencia, 1))
        scores = self.scoring_function(hidden_repr).squeeze(-1)
        # Convierte las puntuaciones en probabilidades
        probabilities = torch.softmax(scores, dim=-1)
        # Muestrea un vector por secuencia basado en las puntuaciones
        selected_indices = torch.multinomial(probabilities, 1).squeeze(-1)
        # Selecciona los vectores basados en los índices muestreados
        batch_size, seq_len, _ = sequence.shape
        selected_vectors = sequence[torch.arange(batch_size), selected_indices]

        return selected_vectors

# Ejemplo de uso
batch_size, seq_len, input_dim, hidden_dim = 32, 10, 100, 50
sequence = torch.randn(batch_size, seq_len, input_dim)

# Instancia el modelo
hard_attention = HardAttention(input_dim, hidden_dim)
selected_vectors = hard_attention(sequence)

print(selected_vectors.shape)  # Forma esperada : (batch_size, input_dim)



torch.Size([32, 100])


In [22]:
sentences = [
    "El usuario no puede acceder a la red",
    "Las computadoras no pueden conectarse a la red",
    "El servidor de correo esta funcionando correctamente",
    "La laptop no puede cargar la bateria"
]

# Funcion ficticia para convertir palabras en embeddings
def word_to_embedding(word):
    # Para simplificar, esta función genera un vector aleatorio para cada palabra.
    # En la práctica, usarías embeddings de palabras reales.
    return torch.randn(100)

# Convertir oraciones a secuencias de embeddings
sequences = [torch.stack([word_to_embedding(word) for word in sentence.split()]) for sentence in sentences]


In [28]:
# Inicializar el modelo de atención dura
input_dim = 100  # Dimensión de los embeddings de palabras
hidden_dim = 50  # Dimensión oculta para la capa intermedia en el modelo
hard_attention = HardAttention(input_dim, hidden_dim)

# Procesar cada oración y seleccionar la palabra más importante
for i, sequence in enumerate(sequences):
    # Agregar una dimensión de lote (tamaño de lote = 1)
    sequence = sequence.unsqueeze(0)
    # Usar el modelo de atención dura para seleccionar una palabra
    selected_vector = hard_attention(sequence)
    # Eliminar la dimensión de lote
    selected_vector = selected_vector.squeeze(0)
    
    # Para demostración, solo imprimiremos el índice de la palabra seleccionada
    # En una aplicación real, mapearías esto de vuelta a la palabra real
    word_index = torch.argmin(torch.sum((sequence.squeeze(0) - selected_vector) ** 2, dim=1))
    print(f"Sentence: '{sentences[i]}'")
    print(f"Summarized by selecting the word: '{sentences[i].split()[word_index.item()]}'\n")


Sentence: 'El usuario no puede acceder a la red'
Summarized by selecting the word: 'la'

Sentence: 'Las computadoras no pueden conectarse a la red'
Summarized by selecting the word: 'a'

Sentence: 'El servidor de correo esta funcionando correctamente'
Summarized by selecting the word: 'funcionando'

Sentence: 'La laptop no puede cargar la bateria'
Summarized by selecting the word: 'cargar'



### Implementando Hard Attention con Numpy

In [44]:
import numpy as np 

def initialize_parameters( input_dim ):
  np.random.seed(42) # Seed para reproducibilidad
  weights = np.random.randn( input_dim, 1 ) # Inicialización aleatoria de los pesos
  bias = np.random.randn() # Inicialización aleatoria del sesgo
  return weights, bias

def compute_scores( sequence, weights, bias ):
  """
  Calcular las puntuaciones para cada vector en la secuencia
  sequence: Arreglo Numpy de forma ( seq_len, input_dim )
  weights: Arreglo Numpy de forma ( input_dim, 1 )
  bias: Escalar (flotante)
  """
  scores = np.dot( sequence, weights ) + bias
  return scores.squeeze()

def select_vector( scores, sequence ):
  """
  Selecciona un vector con la puntuación más alta
  scores: Arreglo Numpy de forma ( seq_len, )
  sequence: Arreglo Numpy de forma ( seq_len, input_dim )
  """
  selected_index = np.argmax( scores ) # Índice del vector seleccionado
  selected_vector = sequence[ selected_index ] # Vector seleccionado
  return selected_vector


# Ejemplo de secuencia de 5 vectores de 3 dimensiones
sequence = np.array(
  [
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0],
    [13.0, 14.0, 15.0]
  ]
)

input_dim = 3 # Dimensión de los vectores de entrada
weights, bias = initialize_parameters( input_dim ) # Inicializar los parámetros

scores = compute_scores( sequence, weights, bias ) # Calcular las puntuaciones
selected_vector = select_vector( scores, sequence ) # Seleccionar el vector con el puntaje más alto

print( "Vector seleccionado:", scores )

Vector seleccionado: [ 3.68628102  6.70469619  9.72311136 12.74152653 15.7599417 ]


In [45]:
sentences = [
  "Yo amo la programación en Python",
  "El clima esta demasiado soleado",
  "El beisbol es un deporte popular en Estados Unidos",
  "La música es una forma de arte", 
  "El aprendizaje automático es fascinante"
]

# Simular la word embedding asignando cada palabra unica un entero ID 
word_to_id = {
  "yo": 1,
  "amo": 2,
  "la": 3,
  "programación": 4,
  "en": 5,
  "python": 6,
  "el": 7,
  "clima": 8,
  "esta": 9,
  "demasiado": 10,
  "soleado": 11,
  "beisbol": 12,
  "es": 13,
  "un": 14,
  "deporte": 15,
  "popular": 16,
  "estados": 17,
  "unidos": 18,
  "música": 19,
  "una": 20,
  "forma": 21,
  "de": 22,
  "arte": 23,
  "aprendizaje": 24,
  "automático": 25,
  "fascinante": 26
}

# Convertir oraciones a secuencias de embeddings
sentence_ids = [[word_to_id[word.lower()] for word in sentence.split()] for sentence in sentences]

# Definir manualmente el puntaje para cada ID de palabra ( puntaje más alto indica una mayor importancia )
word_scores = {
  1: 0.8, 2: 0.7, 3: 0.6, 4: 0.9, 5: 0.5, 6: 0.8,
  7: 0.6, 8: 0.4, 9: 0.3, 10: 0.4, 11: 0.2,
  12: 0.7, 13: 0.6, 14: 0.5, 15: 0.6, 16: 0.4,
  17: 0.5, 18: 0.4, 19: 0.7, 20: 0.6, 21: 0.5,
  22: 0.4, 23: 0.8, 24: 0.9, 25: 0.7, 26: 0.8
}

# Asignar una categoria a cada oración basada en la palabra con el puntaje más alto
sentence_categories = {
  "programación": "Tecnología", 
  "soleado": "Clima",
  "beisbol": "Deportes",
  "música": "Arte",
  "aprendizaje": "Tecnología"
}

In [46]:
def select_important_word( sentence, word_scores ):
  # Convertir la secuencia a puntajes usando el word_scores predefinido
  sentence_scores = [word_scores[word_id] for word_id in sentence]

  # Usar el mecanismo de selección de Hard Attention
  important_word_id = select_vector( np.array( sentence_scores ), np.array( sentence ) )

  # Encontrar la palabra correspondiente al id de important_word_id
  important_word = [ word for word, id in word_to_id.items() if id == important_word_id ][0]
  return important_word

# Clasificar cada oración en una categoría basada en la palabra más importante
for i, sentence_id in enumerate(sentence_ids):
  important_word = select_important_word(sentence_id, word_scores)
  category = sentence_categories.get( important_word, "Desconocido" )
  print( f"Oración: '{sentences[i]}'")
  print( f"Palabra importante: '{important_word}', Categoría: '{category}'\n" )

Oración: 'Yo amo la programación en Python'
Palabra importante: 'programación', Categoría: 'Tecnología'

Oración: 'El clima esta demasiado soleado'
Palabra importante: 'el', Categoría: 'Desconocido'

Oración: 'El beisbol es un deporte popular en Estados Unidos'
Palabra importante: 'beisbol', Categoría: 'Deportes'

Oración: 'La música es una forma de arte'
Palabra importante: 'arte', Categoría: 'Desconocido'

Oración: 'El aprendizaje automático es fascinante'
Palabra importante: 'aprendizaje', Categoría: 'Tecnología'

